# Intervals tutorial

#### Sections
* [Setup](#Setup)
* [Intervals](#Intervals)
* [Invalid Intervals](#Invalid-Intervals)
    * [Unit limits](#Unit-limits)
    * [Data availability](#Data-availability)
* [Overlapping Intervals](#Overlapping-Intervals)
* [Inferred Intervals](#Inferred-Intervals)

#### Notes
* In this tutorial **period parameters** are passed to the `get` method to define the period over which prices are required. These parameters and the evaluaton of the period are covered in the [periods](#./periods.ipynb) tutorial.
* The cell **outputs** shown in this tutorial are based on executing the cells at the time shown in the output of the following cell. Simply rerun the cells to bring any dynamic output up to date.

In [2]:
import pandas as pd
from zoneinfo import ZoneInfo
now = pd.Timestamp.now(tz=ZoneInfo("UTC")).floor("T")
print(f"{now!r}")
print(f"{now.astimezone(ZoneInfo('America/New_York'))!r}")

Timestamp('2022-05-13 16:27:00+0000', tz='UTC')
Timestamp('2022-05-13 12:27:00-0400', tz='America/New_York')


## Setup

Run the following cell to import the tutorial dependencies.

In [3]:
from market_prices import PricesYahoo
import pandas as pd
from market_prices.support import tutorial_helpers as th

Run the following cell to instantiate prices objects and define values used in this tutorial.

In [4]:
prices = PricesYahoo("MSFT")  # prices for US stock Microsoft
prices_btc = PricesYahoo("BTC-USD")  # prices for bitcoin denominated in US dollars
oob_T2_start, oob_T2_end = th.get_sessions_range_for_bi(prices, prices.bis.T5)

## Intervals

`market_prices` returns price data as pandas DataFrame objects.

In [5]:
prices.get("5min", minutes=30)

symbol                                            MSFT              \
                                                  open        high   
[2022-05-13 12:00:00, 2022-05-13 12:05:00)  261.299988  261.619995   
[2022-05-13 12:05:00, 2022-05-13 12:10:00)  261.589996  262.029999   
[2022-05-13 12:10:00, 2022-05-13 12:15:00)  262.024994  262.369995   
[2022-05-13 12:15:00, 2022-05-13 12:20:00)  262.311493  262.380005   
[2022-05-13 12:20:00, 2022-05-13 12:25:00)  261.949890  262.169708   
[2022-05-13 12:25:00, 2022-05-13 12:30:00)  261.714996  261.730011   

symbol                                                                      
                                                   low       close  volume  
[2022-05-13 12:00:00, 2022-05-13 12:05:00)  261.149994  261.600006  180121  
[2022-05-13 12:05:00, 2022-05-13 12:10:00)  261.369995  262.019989  288140  
[2022-05-13 12:10:00, 2022-05-13 12:15:00)  261.790009  262.263702  178641  
[2022-05-13 12:15:00, 2022-05-13 12:20:00)  261.929993  261.950012  194488  
[2022-05-13 12:20:00, 2022-05-13 12:25:00)  261.559998  261.690002  244387  
[2022-05-13 12:25:00, 2022-05-13 12:30:00)  261.114990  261.355011  100788

The DataFrame have 'open', 'high', 'low', 'close' and 'volume' columns for each symbol for which prices have been requested. Each row represents the prices over a specific interval of time, such that:
* 'open' and 'close' columns give the price at, respectively, the start and end of the interval.
* 'high' and 'low' columns give the, respectively, highest and lowest price registered during the interval.
* 'volume' column gives the total volume traded over the interval.

The required `interval` can be defined as a string comprising a value, as one or more digits, followed by one of the following units:
* "min", "t", "MIN" or "T" for minutes, e.g. "5T"
* "h" or "H" for hours, e.g. "3H"
* "d" or "D' for days, e.g. "3D"
* 'm' or "M" for months, e.g. "2M"

The `interval` is specified by the first argument to `get`.

In [6]:
prices.get("1H", days=1)  # one trading day of data at 1 hour intervals

symbol                                            MSFT              \
                                                  open        high   
[2022-05-12 12:30:00, 2022-05-12 13:30:00)  254.839996  256.410004   
[2022-05-12 13:30:00, 2022-05-12 14:30:00)  253.097900  253.550003   
[2022-05-12 14:30:00, 2022-05-12 15:30:00)  250.949997  254.199997   
[2022-05-12 15:30:00, 2022-05-12 16:30:00)  252.460007  255.779999   
[2022-05-13 09:30:00, 2022-05-13 10:30:00)  258.250000  262.089996   
[2022-05-13 10:30:00, 2022-05-13 11:30:00)  261.299988  261.579987   
[2022-05-13 11:30:00, 2022-05-13 12:30:00)  261.230011  262.380005   

symbol                                                                       
                                                   low       close   volume  
[2022-05-12 12:30:00, 2022-05-12 13:30:00)  252.389999  253.100006  4579244  
[2022-05-12 13:30:00, 2022-05-12 14:30:00)  250.220001  250.975006  5689406  
[2022-05-12 14:30:00, 2022-05-12 15:30:00)  250.020004  252.470001  5883838  
[2022-05-12 15:30:00, 2022-05-12 16:30:00)  252.410004  255.369995  5822703  
[2022-05-13 09:30:00, 2022-05-13 10:30:00)  255.350006  261.279999  9855422  
[2022-05-13 10:30:00, 2022-05-13 11:30:00)  259.670013  261.230103  3683204  
[2022-05-13 11:30:00, 2022-05-13 12:30:00)  260.709991  261.355011  2619665

In [7]:
prices.get("13T", hours=2)  # last 2 hours of data at 13 minute intervals

symbol                                            MSFT              \
                                                  open        high   
[2022-05-13 10:35:00, 2022-05-13 10:48:00)  260.739990  261.579987   
[2022-05-13 10:48:00, 2022-05-13 11:01:00)  260.619995  261.149994   
[2022-05-13 11:01:00, 2022-05-13 11:14:00)  260.510010  261.100006   
[2022-05-13 11:14:00, 2022-05-13 11:27:00)  260.605011  261.140015   
[2022-05-13 11:27:00, 2022-05-13 11:40:00)  261.010010  261.480011   
[2022-05-13 11:40:00, 2022-05-13 11:53:00)  261.260010  261.880005   
[2022-05-13 11:53:00, 2022-05-13 12:06:00)  261.725006  261.915009   
[2022-05-13 12:06:00, 2022-05-13 12:19:00)  261.600006  262.380005   
[2022-05-13 12:19:00, 2022-05-13 12:32:00)  262.100006  262.169708   

symbol                                                                        
                                                   low       close    volume  
[2022-05-13 10:35:00, 2022-05-13 10:48:00)  260.269989  260.660004  940955.0  
[2022-05-13 10:48:00, 2022-05-13 11:01:00)  260.331085  260.614990  679253.0  
[2022-05-13 11:01:00, 2022-05-13 11:14:00)  260.079987  260.609985  759449.0  
[2022-05-13 11:14:00, 2022-05-13 11:27:00)  259.670013  260.980011  854741.0  
[2022-05-13 11:27:00, 2022-05-13 11:40:00)  260.709991  261.265015  657882.0  
[2022-05-13 11:40:00, 2022-05-13 11:53:00)  260.721100  261.725006  582846.0  
[2022-05-13 11:53:00, 2022-05-13 12:06:00)  261.140015  261.589905  630014.0  
[2022-05-13 12:06:00, 2022-05-13 12:19:00)  261.500000  262.105011  518508.0  
[2022-05-13 12:19:00, 2022-05-13 12:32:00)  261.114990  261.355011  389444.0

If only `interval` is passed then all available price history at that interval is returned.

In [8]:
df = prices.get("4H")  # all available data at 4H intervals
df

symbol                                            MSFT              \
                                                  open        high   
[2020-05-13 13:30:00, 2020-05-13 17:30:00)  178.570007  180.220001   
[2020-05-14 09:30:00, 2020-05-14 13:30:00)  177.539993  178.889999   
[2020-05-14 13:30:00, 2020-05-14 17:30:00)  178.100006  180.690002   
[2020-05-15 09:30:00, 2020-05-15 13:30:00)  179.059998  182.000000   
[2020-05-15 13:30:00, 2020-05-15 17:30:00)  180.470001  187.059998   
...                                                ...         ...   
[2022-05-11 09:30:00, 2022-05-11 13:30:00)  265.679993  271.070007   
[2022-05-11 13:30:00, 2022-05-11 17:30:00)  263.277191  264.427887   
[2022-05-12 09:30:00, 2022-05-12 13:30:00)  257.690002  259.880005   
[2022-05-12 13:30:00, 2022-05-12 17:30:00)  253.097900  255.779999   
[2022-05-13 09:30:00, 2022-05-13 13:30:00)  258.250000  262.380005   

symbol                                                                          
                                                   low       close      volume  
[2020-05-13 13:30:00, 2020-05-13 17:30:00)  177.464996  179.740005  12392011.0  
[2020-05-14 09:30:00, 2020-05-14 13:30:00)  175.679993  178.109894  22767697.0  
[2020-05-14 13:30:00, 2020-05-14 17:30:00)  177.020004  180.460007  11564399.0  
[2020-05-15 09:30:00, 2020-05-15 13:30:00)  177.130005  180.460007  20972152.0  
[2020-05-15 13:30:00, 2020-05-15 17:30:00)  179.649994  183.350494  16280621.0  
...                                                ...         ...         ...  
[2022-05-11 09:30:00, 2022-05-11 13:30:00)  262.269989  263.306000  21432202.0  
[2022-05-11 13:30:00, 2022-05-11 17:30:00)  259.299988  260.660004  16421843.0  
[2022-05-12 09:30:00, 2022-05-12 13:30:00)  252.389999  253.100006  27567859.0  
[2022-05-12 13:30:00, 2022-05-12 17:30:00)  250.020004  255.369995  17395947.0  
[2022-05-13 09:30:00, 2022-05-13 13:30:00)  255.350006  261.355011  16158291.0  

[1007 rows x 5 columns]

The `DataFrame` is indexed with an `IntervalIndex`, with each row represented by a pandas `Interval`.

In [9]:
indice = df.index[-2]
indice

Interval('2022-05-12 13:30:00', '2022-05-12 17:30:00', closed='left')

Each interval is closed on the left such that the period that a row covers is inclusive of the interval's left side but exclusive of the interval's right side.

In [10]:
indice.left, indice.right

(Timestamp('2022-05-12 13:30:00-0400', tz='America/New_York'),
 Timestamp('2022-05-12 17:30:00-0400', tz='America/New_York'))

By default all intraday intervals will be of equal length and reflect the passed `interval`.

* The [Overlapping Intervals](#Overlapping-Intervals) section explains how some intraday intervals can be irregular when intervals would otherwise overlap.
* The [anchor](./anchor.ipynb) tutorial explains how intraday intervals can be irregular when the `anchor` option is passed as 'workback'.
* The [anchor](./anchor.ipynb) tutorial also covers the `openend` and `force` options which can result in an irregular interval at, respectively, the end of the period and the end of each session and/or subsession.

Daily price data is indexed differently to intraday data. The pandas `DatetimeIndex` provides for each row to be indexed with a single timestamp that represents a daily session.

In [11]:
df_daily = prices.get("1D", days=6)
df_daily

symbol            MSFT                                              
                  open        high         low       close    volume
2022-05-06  274.809998  279.250000  271.269989  274.730011  37748300
2022-05-09  270.059998  272.359985  263.320007  264.579987  47726000
2022-05-10  271.690002  273.750000  265.070007  269.500000  39336400
2022-05-11  265.679993  271.359985  259.299988  260.549988  48975900
2022-05-12  257.690002  259.880005  250.020004  255.350006  50970700
2022-05-13  257.350006  262.380005  255.350006  261.450012  16248281

If the interval is a multiple of days then the index is again represented by a pandas `IntervalIndex` and each indice by an Interval.

In [12]:
prices.get("2D", days=6)

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-05-06, 2022-05-10)  274.809998  279.250000  263.320007  264.579987   
[2022-05-10, 2022-05-12)  271.690002  273.750000  259.299988  260.549988   
[2022-05-12, 2022-05-14)  257.690002  262.380005  250.020004  261.450012   

symbol                                
                              volume  
[2022-05-06, 2022-05-10)  85474300.0  
[2022-05-10, 2022-05-12)  88312300.0  
[2022-05-12, 2022-05-14)  67218981.0

The same principle applies as for the intraday data - the data for each of the above indices is inclusive of the session represented by the interval's left side but exclusive of any session that might be represented by the interval's right side.

In [13]:
prices.get("2M", years=1)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-06-01, 2021-08-01)  251.229996  290.149994  243.000000  284.910004   
[2021-08-01, 2021-10-01)  286.359985  305.839996  281.619995  281.920013   
[2021-10-01, 2021-12-01)  282.119995  349.670013  280.250000  330.589996   
[2021-12-01, 2022-02-01)  335.130005  344.299988  276.049988  310.980011   
[2022-02-01, 2022-04-01)  310.410004  315.950012  270.000000  308.309998   
[2022-04-01, 2022-06-01)  309.369995  315.109985  250.020004  261.450012   

symbol                                  
                                volume  
[2021-06-01, 2021-08-01)  1.031245e+09  
[2021-08-01, 2021-10-01)  9.442276e+08  
[2021-10-01, 2021-12-01)  1.026401e+09  
[2021-12-01, 2022-02-01)  1.573206e+09  
[2022-02-01, 2022-04-01)  1.431385e+09  
[2022-04-01, 2022-06-01)  1.006289e+09

`interval` can alternatively be passed as a pandas `Timedelta` object.

In [14]:
interval = pd.Timedelta(30, "T")
prices.get(interval, hours=1, minutes=30)

symbol                                            MSFT              \
                                                  open        high   
[2022-05-13 11:00:00, 2022-05-13 11:30:00)  260.769989  261.380005   
[2022-05-13 11:30:00, 2022-05-13 12:00:00)  261.230011  261.915009   
[2022-05-13 12:00:00, 2022-05-13 12:30:00)  261.299988  262.380005   

symbol                                                                         
                                                   low       close     volume  
[2022-05-13 11:00:00, 2022-05-13 11:30:00)  259.670013  261.230103  1807887.0  
[2022-05-13 11:30:00, 2022-05-13 12:00:00)  260.709991  261.309998  1433100.0  
[2022-05-13 12:00:00, 2022-05-13 12:30:00)  261.114990  261.355011  1186565.0

In [15]:
interval = pd.Timedelta(2, "D")
prices.get(interval, days=6)

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-05-06, 2022-05-10)  274.809998  279.250000  263.320007  264.579987   
[2022-05-10, 2022-05-12)  271.690002  273.750000  259.299988  260.549988   
[2022-05-12, 2022-05-14)  257.690002  262.380005  250.020004  261.450012   

symbol                                
                              volume  
[2022-05-06, 2022-05-10)  85474300.0  
[2022-05-10, 2022-05-12)  88312300.0  
[2022-05-12, 2022-05-14)  67218981.0

## Invalid Intervals

### Interval longer than period
The interval cannot be higher than the period over which prices are being requested.

In [ ]:
prices.get("2H", hours=1, minutes=30)

```
---------------------------------------------------------------------------
PricesUnavailableIntervalDurationError    Traceback (most recent call last)
<ipython-input-16-e088d3dea4ce> in <module>
----> 1 prices.get("2H", hours=1, minutes=30)

PricesUnavailableIntervalDurationError: Period duration shorter than interval. Interval is 0 days 02:00:00 although period duration is only 0 days 01:30:00.
Duration evaluated from parameters: {'minutes': 30, 'hours': 1, 'days': 0, 'weeks': 0, 'months': 0, 'years': 0, 'start': None, 'end': None, 'add_a_row': False}.
```

### Unit limits
For each unit there is maximum value that cannot be exceeded:
* minutes and hours: equivalent of 22 hours or 1320 minutes
* days: 250
* months: 36

In [ ]:
prices.get("23H", months=1)

```
---------------------------------------------------------------------------
ValidationError                           Traceback (most recent call last)
<ipython-input-69-5eaeb06401af> in <module>
----> 1 prices.get("23H", months=1)

ValidationError: 1 validation error for Get
interval
  An `interval` defined in terms of hours cannot have a value greater than 22, although received `interval` as "23H". (type=value_error)
```

### Data availability

Prices can only be provided at a specific interval if underlying data is available either at that interval or at a lower interval from which the data can be downsampled to the requested interval.

Data availability is covered on the dedicated [data availability](./data_availability.ipynb) tutorial. For the purposes of this tutorial, suffice to say that if an intraday interval is requested for which underlying data is not available, a `PricesIntradayUnavailableError` error is raised.

In [18]:
start, end = oob_T2_start, oob_T2_end
start, end

(Timestamp('2022-03-15 00:00:00', freq='C'),
 Timestamp('2022-03-30 00:00:00', freq='C'))

In [ ]:
prices.get("2T", start=start, end=end)

```
---------------------------------------------------------------------------
PricesIntradayUnavailableError            Traceback (most recent call last)
<ipython-input-19-8e6dfbd453e2> in <module>
----> 1 prices.get("2T", start=start, end=end)

PricesIntradayUnavailableError: Data is unavailable at a sufficiently low base interval to evaluate prices at interval 0 days 00:02:00 anchored 'Anchor.OPEN'.
Base intervals that are a factor of 0 days 00:02:00:
	[<BaseInterval.T1: Timedelta('0 days 00:01:00')>, <BaseInterval.T2: Timedelta('0 days 00:02:00')>].
The earliest minute from which data is available at 0 days 00:02:00 is 2022-03-31 16:29:00+00:00, although at this base interval the requested period evaluates to (Timestamp('2022-03-15 13:30:00+0000', tz='UTC'), Timestamp('2022-03-30 20:00:00+0000', tz='UTC')).
Period evaluated from parameters: {'minutes': 0, 'hours': 0, 'days': 0, 'weeks': 0, 'months': 0, 'years': 0, 'start': Timestamp('2022-03-15 00:00:00', freq='C'), 'end': Timestamp('2022-03-30 00:00:00', freq='C'), 'add_a_row': False}.
```

## Overlapping Intervals

If the `interval` is longer than the gap between the end of one session and the start of the next, or longer than the length of a session break, then it's possible for the last interval of a (sub)session to overlap with the first interval of the next (sub)session. In this case the right side of the last (sub)session interval is curtailed to the start of the next.

Bitcoin trades 24/7 with sessions assumed to start at midnight UTC. If the interval is 9 hours then, as 9 is not a factor of 24, the last interval of each session would overlap with the first interval of the next.

In [ ]:
df = prices_btc.get("9H", days=2)

```
IntervalIrregularWarning: 
PriceTable interval is irregular. One or more indices were curtailed to prevent the last indice assigned to a (sub)session from overlapping with the first indice of the following (sub)session.
Use .pt.indices_length and .pt.by_indice_length to interrogate.
  warnings.warn(errors.IntervalIrregularWarning())
```

In [18]:
df

symbol                                           BTC-USD                \
                                                    open          high   
[2022-05-10 18:00:00, 2022-05-11)           31662.894531  31747.683594   
[2022-05-11, 2022-05-11 09:00:00)           31020.472656  31732.447266   
[2022-05-11 09:00:00, 2022-05-11 18:00:00)  31197.576172  31988.117188   
[2022-05-11 18:00:00, 2022-05-12)           29838.402344  30154.968750   
[2022-05-12, 2022-05-12 09:00:00)           28936.734375  29989.203125   
[2022-05-12 09:00:00, 2022-05-12 18:00:00)  27819.166016  29761.673828   

symbol                                                                  \
                                                     low         close   
[2022-05-10 18:00:00, 2022-05-11)           30320.798828  31012.925781   
[2022-05-11, 2022-05-11 09:00:00)           30330.562500  31167.394531   
[2022-05-11 09:00:00, 2022-05-11 18:00:00)  29323.230469  29832.398438   
[2022-05-11 18:00:00, 2022-05-12)           28170.412109  28936.253906   
[2022-05-12, 2022-05-12 09:00:00)           26350.490234  27804.671875   
[2022-05-12 09:00:00, 2022-05-12 18:00:00)  27608.849609  28670.439453   

symbol                                                    
                                                  volume  
[2022-05-10 18:00:00, 2022-05-11)           1.264677e+09  
[2022-05-11, 2022-05-11 09:00:00)           4.717359e+09  
[2022-05-11 09:00:00, 2022-05-11 18:00:00)  1.753438e+10  
[2022-05-11 18:00:00, 2022-05-12)           5.823877e+09  
[2022-05-12, 2022-05-12 09:00:00)           1.621221e+10  
[2022-05-12 09:00:00, 2022-05-12 18:00:00)  1.072715e+10

In this case a warning is raised to advise that some intervals were curtailed and hence that the interval should not be considered as regular.

## Inferred Intervals

The `interval` parameter is optional. If not passed then the interval will be inferred from any 'period parameters' (passed to define the period over which prices are being requested).

The interval will be inferred as an intraday interval whenever:
* The period duration is defined in terms of `minutes` and/or `hours`.
* The period duration is defined in terms of `days` and the value is 5 or less.
* The period is defined with `start` and `end` and either is passed as a time (as opposed to a date).
* The period is defined with `start` and (optionally) `end` which represent dates that enclose a range of 5 sessions or less.  

In these cases the interval will be inferred as the highest viable intraday base interval available over the requested period.

(Defining the period over which prices are requested is covered by the [periods](./periods.ipyny) tutorial, whilst base intervals are explained in [data availability](./data_availability.ipynb) tutorial.)

In [21]:
# period duration defined in minutes
prices.get(minutes=30)

symbol                                            MSFT              \
                                                  open        high   
[2022-05-13 12:00:00, 2022-05-13 12:05:00)  261.299988  261.619995   
[2022-05-13 12:05:00, 2022-05-13 12:10:00)  261.589996  262.029999   
[2022-05-13 12:10:00, 2022-05-13 12:15:00)  262.024994  262.369995   
[2022-05-13 12:15:00, 2022-05-13 12:20:00)  262.311493  262.380005   
[2022-05-13 12:20:00, 2022-05-13 12:25:00)  261.949890  262.169708   
[2022-05-13 12:25:00, 2022-05-13 12:30:00)  261.714996  261.730011   

symbol                                                                        
                                                   low       close    volume  
[2022-05-13 12:00:00, 2022-05-13 12:05:00)  261.149994  261.600006  180121.0  
[2022-05-13 12:05:00, 2022-05-13 12:10:00)  261.369995  262.019989  288140.0  
[2022-05-13 12:10:00, 2022-05-13 12:15:00)  261.790009  262.263702  178641.0  
[2022-05-13 12:15:00, 2022-05-13 12:20:00)  261.929993  261.950012  194488.0  
[2022-05-13 12:20:00, 2022-05-13 12:25:00)  261.559998  261.690002  244387.0  
[2022-05-13 12:25:00, 2022-05-13 12:30:00)  261.114990  261.500000  159246.0

In [22]:
# period duration defined in hours
df = prices.get(hours=6)
df

symbol                                            MSFT              \
                                                  open        high   
[2022-05-12 13:30:00, 2022-05-12 14:30:00)  253.097900  253.550003   
[2022-05-12 14:30:00, 2022-05-12 15:30:00)  250.949997  254.199997   
[2022-05-12 15:30:00, 2022-05-12 16:30:00)  252.460007  255.779999   
[2022-05-13 09:30:00, 2022-05-13 10:30:00)  258.250000  262.089996   
[2022-05-13 10:30:00, 2022-05-13 11:30:00)  261.299988  261.579987   
[2022-05-13 11:30:00, 2022-05-13 12:30:00)  261.230011  262.380005   

symbol                                                                         
                                                   low       close     volume  
[2022-05-12 13:30:00, 2022-05-12 14:30:00)  250.220001  250.975006  5689406.0  
[2022-05-12 14:30:00, 2022-05-12 15:30:00)  250.020004  252.470001  5883838.0  
[2022-05-12 15:30:00, 2022-05-12 16:30:00)  252.410004  255.369995  5822703.0  
[2022-05-13 09:30:00, 2022-05-13 10:30:00)  255.350006  261.279999  9855422.0  
[2022-05-13 10:30:00, 2022-05-13 11:30:00)  259.670013  261.230103  3683204.0  
[2022-05-13 11:30:00, 2022-05-13 12:30:00)  260.709991  261.500000  2656037.0

In [23]:
end = df.index[3].left
end

Timestamp('2022-05-13 09:30:00-0400', tz='America/New_York')

In [24]:
# end defined as a time
prices.get(end=end, days=15)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-22 09:30:00, 2022-04-22 10:30:00)  281.679993  283.200012   
[2022-04-22 10:30:00, 2022-04-22 11:30:00)  279.589996  281.049713   
[2022-04-22 11:30:00, 2022-04-22 12:30:00)  279.260010  279.519989   
[2022-04-22 12:30:00, 2022-04-22 13:30:00)  277.600006  278.364990   
[2022-04-22 13:30:00, 2022-04-22 14:30:00)  277.260010  277.350006   
...                                                ...         ...   
[2022-05-12 11:30:00, 2022-05-12 12:30:00)  259.040009  259.320007   
[2022-05-12 12:30:00, 2022-05-12 13:30:00)  254.839996  256.410004   
[2022-05-12 13:30:00, 2022-05-12 14:30:00)  253.097900  253.550003   
[2022-05-12 14:30:00, 2022-05-12 15:30:00)  250.949997  254.199997   
[2022-05-12 15:30:00, 2022-05-12 16:30:00)  252.460007  255.779999   

symbol                                                                         
                                                   low       close     volume  
[2022-04-22 09:30:00, 2022-04-22 10:30:00)  278.966705  279.600006  5521496.0  
[2022-04-22 10:30:00, 2022-04-22 11:30:00)  276.029999  279.230011  3112562.0  
[2022-04-22 11:30:00, 2022-04-22 12:30:00)  276.839996  277.625000  2510357.0  
[2022-04-22 12:30:00, 2022-04-22 13:30:00)  276.859985  277.269989  2191531.0  
[2022-04-22 13:30:00, 2022-04-22 14:30:00)  275.570007  276.079987  2487992.0  
...                                                ...         ...        ...  
[2022-05-12 11:30:00, 2022-05-12 12:30:00)  253.777893  254.830002  4884195.0  
[2022-05-12 12:30:00, 2022-05-12 13:30:00)  252.389999  253.100006  4579244.0  
[2022-05-12 13:30:00, 2022-05-12 14:30:00)  250.220001  250.975006  5689406.0  
[2022-05-12 14:30:00, 2022-05-12 15:30:00)  250.020004  252.470001  5883838.0  
[2022-05-12 15:30:00, 2022-05-12 16:30:00)  252.410004  255.369995  5822703.0  

[105 rows x 5 columns]

In [25]:
# start and end as dates that cover 5 sessions.
start_daily = df_daily.index[0]
end_daily = df_daily.index[4]
start_daily, end_daily

(Timestamp('2022-05-06 00:00:00', freq='C'),
 Timestamp('2022-05-12 00:00:00', freq='C'))

In [26]:
df = prices.get(start=start_daily, end=end_daily)
df[:3]  # showing only first three rows

symbol                                            MSFT              \
                                                  open        high   
[2022-05-06 09:30:00, 2022-05-06 10:30:00)  274.804993  277.940002   
[2022-05-06 10:30:00, 2022-05-06 11:30:00)  274.049988  279.250000   
[2022-05-06 11:30:00, 2022-05-06 12:30:00)  278.769989  278.970001   

symbol                                                                         
                                                   low       close     volume  
[2022-05-06 09:30:00, 2022-05-06 10:30:00)  271.360291  274.109985  7753500.0  
[2022-05-06 10:30:00, 2022-05-06 11:30:00)  272.559998  278.750000  5762924.0  
[2022-05-06 11:30:00, 2022-05-06 12:30:00)  275.380005  276.910004  3520158.0

If the period definition infers an intraday interval although intraday data is not available over the requested period then a `LastIndiceInaccurateError` is raised.

In [27]:
end

Timestamp('2022-05-13 09:30:00-0400', tz='America/New_York')

In [ ]:
prices.get(end=end, years=3)

```
---------------------------------------------------------------------------
LastIndiceInaccurateError                 Traceback (most recent call last)
<ipython-input-28-70411e342e12> in <module>
----> 1 prices.get(end=end, years=3)

LastIndiceInaccurateError: Full period not available at any synchronised intraday base interval. The following base intervals could represent the end indice with the greatest possible accuracy although have insufficient data available to cover the full period:
	[<BaseInterval.T1: Timedelta('0 days 00:01:00')>, <BaseInterval.T2: Timedelta('0 days 00:02:00')>, <BaseInterval.T5: Timedelta('0 days 00:05:00')>, <BaseInterval.H1: Timedelta('0 days 01:00:00')>].
The earliest minute from which data is available at 0 days 01:00:00 is 2020-05-13 16:30:00+00:00, although at this base interval the requested period evaluates to (Timestamp('2019-05-13 13:30:00+0000', tz='UTC'), Timestamp('2022-05-12 20:30:00+0000', tz='UTC')).
Period evaluated from parameters: {'minutes': 0, 'hours': 0, 'days': 0, 'weeks': 0, 'months': 0, 'years': 3, 'start': None, 'end': Timestamp('2022-05-13 13:30:00+0000', tz='UTC'), 'add_a_row': False}.
Data that can express the period end with the greatest possible accuracy is available from 2020-05-13 16:30:00+00:00. Pass `strict` as False to return prices for this part of the period.
Alternatively, consider creating a composite table (pass `composite` as True) or passing `priority` as 'period'.
```

See the [data_availability](data_availability.ipynb) tutorial for useful options to handle unavailable underlying data (`priority`, `composite` and `strict`).

If the interval isn't inferred as intraday then daily data is returned.

In [29]:
prices.get(days=6)

symbol            MSFT                                              
                  open        high         low       close    volume
2022-05-06  274.809998  279.250000  271.269989  274.730011  37748300
2022-05-09  270.059998  272.359985  263.320007  264.579987  47726000
2022-05-10  271.690002  273.750000  265.070007  269.500000  39336400
2022-05-11  265.679993  271.359985  259.299988  260.549988  48975900
2022-05-12  257.690002  259.880005  250.020004  255.350006  50970700
2022-05-13  257.350006  262.380005  255.350006  261.529999  16275122

In [30]:
# start and end as dates that cover >5 sessions.
end_daily = df_daily.index[5]
start_daily, end_daily

(Timestamp('2022-05-06 00:00:00', freq='C'),
 Timestamp('2022-05-13 00:00:00', freq='C'))

In [31]:
prices.get(start=start_daily, end=end_daily)

symbol            MSFT                                              
                  open        high         low       close    volume
2022-05-06  274.809998  279.250000  271.269989  274.730011  37748300
2022-05-09  270.059998  272.359985  263.320007  264.579987  47726000
2022-05-10  271.690002  273.750000  265.070007  269.500000  39336400
2022-05-11  265.679993  271.359985  259.299988  260.549988  48975900
2022-05-12  257.690002  259.880005  250.020004  255.350006  50970700
2022-05-13  257.350006  262.380005  255.350006  261.529999  16275122

If no arguments are passed then all available daily price history is returned.

In [32]:
prices.get()

symbol            MSFT                                                
                  open        high         low       close      volume
1986-03-13    0.088542    0.101563    0.088542    0.097222  1031788800
1986-03-14    0.097222    0.102431    0.097222    0.100694   308160000
1986-03-17    0.100694    0.103299    0.100694    0.102431   133171200
1986-03-18    0.102431    0.103299    0.098958    0.099826    67766400
1986-03-19    0.099826    0.100694    0.097222    0.098090    47894400
...                ...         ...         ...         ...         ...
2022-05-09  270.059998  272.359985  263.320007  264.579987    47726000
2022-05-10  271.690002  273.750000  265.070007  269.500000    39336400
2022-05-11  265.679993  271.359985  259.299988  260.549988    48975900
2022-05-12  257.690002  259.880005  250.020004  255.350006    50970700
2022-05-13  257.350006  262.380005  255.350006  261.529999    16275122

[9118 rows x 5 columns]